In [1]:
import requests
import json
import sys

sys.path.append('..')

# Data Manipulation
import pandas as pd

from utils import *

# Agregados

In [2]:
url = 'https://servicodados.ibge.gov.br/api/v3/agregados'
df_aggregated = make_df(get_request_json(url))
df_aggregated

,id,nome,agregados
0,D5,Áreas Urbanizadas,"[{'id': '8418', 'nome': 'Áreas urbanizadas, Lo..."
1,CL,Cadastro Central de Empresas,"[{'id': '1685', 'nome': 'Unidades locais, empr..."
2,CA,Censo Agropecuário,"[{'id': '1005', 'nome': 'Número de estabelecim..."
3,ME,"Censo Comum do Mercosul, Bolívia e Chile","[{'id': '1221', 'nome': 'População residente, ..."
4,CD,Censo Demográfico,"[{'id': '102', 'nome': 'Mulheres de 10 anos ou..."
...,...,...,...
63,VS,Produção da Extração Vegetal e da Silvicultura,"[{'id': '289', 'nome': 'Quantidade produzida e..."
64,PO,Produção de Ovos de Galinha,"[{'id': '6672', 'nome': 'Quantidade de ovos pr..."
65,IO,Produto Interno Bruto dos Municípios,"[{'id': '21', 'nome': 'Produto interno bruto a..."
66,XE,Projeção da População,"[{'id': '7358', 'nome': 'População, por sexo e..."


In [3]:
research_ids = df_aggregated['id'].unique()
print(list(research_ids))

['D5', 'CL', 'CA', 'ME', 'CD', 'CM', 'DU', 'DT', 'C4', 'DR', 'DZ', 'SU', 'ST', 'DE', 'D9', 'XF', 'FP', 'IU', 'IL', 'IR', 'IJ', 'PC', 'IA', 'IQ', 'IG', 'PG', 'LA', 'C2', 'AC', 'PB', 'PY', 'PP', 'AM', 'ES', 'PM', 'IT', 'OF', 'SH', 'SE', 'PT', 'RC', 'PI', 'PK', 'PJ', 'DG', 'PZ', 'XY', 'AB', 'MC', 'PE', 'SC', 'PL', 'SB', 'XN', 'AA', 'PD', 'B5', 'BB', 'DD', 'AX', 'CQ', 'LT', 'PA', 'VS', 'PO', 'IO', 'XE', 'SI']


In [4]:
%%time
df_researches = pd.DataFrame()

research_ids = df_aggregated['id'].unique()

for research_id in research_ids:
    # get just the row for the given research id
    df_research = df_aggregated[df_aggregated['id'] == research_id]
    # explode the row to get all the agregados 
    df_research = df_research.explode('agregados')
    # reset index
    df_research = df_research.reset_index()
    # normalize the agregados column and rename the columns
    df_normalize = pd.json_normalize(df_research['agregados'])
    df_normalize = df_normalize.rename(columns = {'id': 'id-agregado', 'nome': 'nome-agregado'})
    # concat the agregado data to df_research
    df_research = pd.concat([df_research, df_normalize], axis = 1)
    # delete the dict column
    df_research = df_research.drop('agregados', axis = 1)
    # rename the columns
    df_research = df_research.rename(columns = {'id': 'id-pesquisa', 'nome': 'nome-pesquisa'})
    # concat the research data to df_researches
    df_researches = pd.concat([df_researches, df_research], axis = 0)
# reset index
df_researches = df_researches.reset_index(drop = True)

CPU times: total: 422 ms
Wall time: 876 ms


In [5]:
df_researches = df_researches[df_researches['id-pesquisa'] == 'CD']
df_researches

,index,id-pesquisa,nome-pesquisa,id-agregado,nome-agregado
1452,4,CD,Censo Demográfico,102,"Mulheres de 10 anos ou mais de idade, que vivi..."
1453,4,CD,Censo Demográfico,103,"Mulheres de 10 anos ou mais de idade, que vivi..."
1454,4,CD,Censo Demográfico,105,Filhos tidos nascidos vivos das mulheres de 10...
1455,4,CD,Censo Demográfico,107,"Mulheres de 10 anos ou mais de idade, que tive..."
1456,4,CD,Censo Demográfico,108,"Mulheres de 10 anos ou mais de idade, resident..."
...,...,...,...,...,...
2385,4,CD,Censo Demográfico,9725,Domicílios particulares permanentes ocupados l...
2386,4,CD,Censo Demográfico,9726,Domicílios particulares permanentes ocupados c...
2387,4,CD,Censo Demográfico,9727,Domicílios particulares permanentes ocupados e...
2388,4,CD,Censo Demográfico,9728,Domicílios particulares permanentes ocupados e...


In [6]:
df_researches['nome-agregado'].unique()

array(['Mulheres de 10 anos ou mais de idade, que viviam em união conjugal, total e que tiveram filhos, e os filhos tidos pelas mulheres de 10 anos ou mais de idade, que viviam em união conjugal, os nascidos vivos, os nascidos vivos no período de referência de 12 meses, nascidos mortos e os que estavam vivos na data de referência, segundo a situação do domicílio, a natureza da união conjugal e os grupos de idade das mulheres',
       'Mulheres de 10 anos ou mais de idade, que viviam em união conjugal e tiveram filhos nascidos vivos, por número de filhos tidos nascidos vivos, segundo a situação do domicílio, a natureza da união conjugal e os grupos de idade das mulheres',
       'Filhos tidos nascidos vivos das mulheres de 10 anos ou mais de idade, que viviam em união conjugal, segundo a situação do domicílio, a natureza da união conjugal e os grupos de idade das mulheres',
       'Mulheres de 10 anos ou mais de idade, que tiveram filhos nascidos vivos, por número de filhos tidos nascid

# Metadados das pesquisas

In [7]:
%%time
df_metadata = pd.DataFrame()

aggregate_ids = df_researches['id-agregado'].unique()

for aggregate_id in aggregate_ids[0:10]:
    # create the url to request the metadata
    url_metadata = f"https://servicodados.ibge.gov.br/api/v3/agregados/{aggregate_id}/metadados"
    # make the request and get the metadata json
    df_aux = make_df([get_request_json(url_metadata)])
    # explode the row to get all the metadata
    # df_aux = df_aux.explode(['metadados'])
    # concat the metadata to df_metadata
    df_metadata = pd.concat([df_metadata, df_aux], axis = 0)
    # convert the floats to int
# reset index
df_metadata = df_metadata.reset_index(drop = True)

CPU times: total: 938 ms
Wall time: 3min 8s


In [8]:
df_metadata.info()
df_metadata.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 15 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   id                               9 non-null      float64
 1   nome                             9 non-null      object 
 2   URL                              9 non-null      object 
 3   pesquisa                         9 non-null      object 
 4   assunto                          9 non-null      object 
 5   periodicidade_frequencia         9 non-null      object 
 6   periodicidade_inicio             9 non-null      float64
 7   periodicidade_fim                9 non-null      float64
 8   nivelTerritorial_Administrativo  9 non-null      object 
 9   nivelTerritorial_Especial        9 non-null      object 
 10  nivelTerritorial_IBGE            9 non-null      object 
 11  variaveis                        9 non-null      object 
 12  classificacoes           

,id,nome,URL,pesquisa,assunto,periodicidade_frequencia,periodicidade_inicio,periodicidade_fim,nivelTerritorial_Administrativo,nivelTerritorial_Especial,nivelTerritorial_IBGE,variaveis,classificacoes,statusCode,message
0,102.0,"Mulheres de 10 anos ou mais de idade, que vivi...",https://sidra.ibge.gov.br/tabela/102,Censo Demográfico,Fecundidade,anual,2010.0,2010.0,"[N15, N1, N2, N8, N9, N6, N14, N7, N13, N3]",[],[],"[{'id': 1488, 'nome': 'Mulheres de 10 anos ou ...","[{'id': 1, 'nome': 'Situação do domicílio', 's...",NaN,NaN
1,103.0,"Mulheres de 10 anos ou mais de idade, que vivi...",https://sidra.ibge.gov.br/tabela/103,Censo Demográfico,Fecundidade,anual,2010.0,2010.0,"[N15, N1, N2, N8, N9, N6, N14, N7, N13, N3]",[],[],"[{'id': 1528, 'nome': 'Mulheres de 10 anos ou ...","[{'id': 12291, 'nome': 'Número de filhos tidos...",NaN,NaN
2,105.0,Filhos tidos nascidos vivos das mulheres de 10...,https://sidra.ibge.gov.br/tabela/105,Censo Demográfico,Fecundidade,anual,2010.0,2010.0,"[N15, N1, N2, N8, N9, N6, N14, N7, N13, N3]",[],[],"[{'id': 1515, 'nome': 'Filhos tidos nascidos v...","[{'id': 1, 'nome': 'Situação do domicílio', 's...",NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,500.0,Internal server error
4,108.0,"Mulheres de 10 anos ou mais de idade, resident...",https://sidra.ibge.gov.br/tabela/108,Censo Demográfico,Fecundidade,anual,2010.0,2010.0,"[N15, N1, N2, N8, N9, N6, N14, N7, N13, N3]",[],[],"[{'id': 1533, 'nome': 'Mulheres de 10 anos ou ...","[{'id': 1, 'nome': 'Situação do domicílio', 's...",NaN,NaN


In [9]:
# checar mais tarde
df_metadata['classificacoes'][2]

[{'id': 1,
  'nome': 'Situação do domicílio',
  'sumarizacao': {'status': True, 'excecao': []},
  'categorias': [{'id': 0, 'nome': 'Total', 'unidade': None, 'nivel': 0},
   {'id': 1, 'nome': 'Urbana', 'unidade': None, 'nivel': 1},
   {'id': 2, 'nome': 'Rural', 'unidade': None, 'nivel': 1}]},
 {'id': 1598,
  'nome': 'Natureza da união conjugal',
  'sumarizacao': {'status': True, 'excecao': []},
  'categorias': [{'id': 0, 'nome': 'Total', 'unidade': None, 'nivel': 0},
   {'id': 99812,
    'nome': 'Casamento civil e religioso',
    'unidade': None,
    'nivel': 1},
   {'id': 99813,
    'nome': 'Somente casamento civil',
    'unidade': None,
    'nivel': 1},
   {'id': 99814,
    'nome': 'Somente casamento religioso',
    'unidade': None,
    'nivel': 1},
   {'id': 99815, 'nome': 'União consensual', 'unidade': None, 'nivel': 1}]},
 {'id': 12232,
  'nome': 'Grupos de idade das mulheres',
  'sumarizacao': {'status': True, 'excecao': []},
  'categorias': [{'id': 0, 'nome': 'Total', 'unidade': 

In [10]:
df_metadata['variaveis'][2]

[{'id': 1515,
  'nome': 'Filhos tidos nascidos vivos pelas mulheres de 10 anos ou mais de idade que viviam em união conjugal',
  'unidade': 'Pessoas',
  'sumarizacao': ['nivelTerritorial']},
 {'id': 1001515,
  'nome': 'Filhos tidos nascidos vivos pelas mulheres de 10 anos ou mais de idade que viviam em união conjugal - percentual do total geral',
  'unidade': '%',
  'sumarizacao': []}]

## Testes

+ para cada agregado
+ para cada nível
+ explodir os resultados
+ para cada variável
+ fazer um df
+ transformar a coluna de série em um dataframe

In [11]:
agregado = 2905
nivel = 'N2' #[N2, N6, N3]	[N23, N132]
url = f'https://servicodados.ibge.gov.br/api/v3/agregados/{agregado}/variaveis?localidades={nivel}[all]' 

In [12]:
get_request_json(url)[0]

{'id': '849',
 'variavel': 'Distribuição do rendimento nominal mensal das pessoas de 10 anos ou mais de idade com rendimento',
 'unidade': '%',
 'resultados': [{'classificacoes': [{'id': '2',
     'nome': 'Sexo',
     'categoria': {'6794': 'Total'}},
    {'id': '12067',
     'nome': 'Classes acumuladas de percentual em ordem crescente de rendimento',
     'categoria': {'99852': 'Até 100 %'}}],
   'series': [{'localidade': {'id': '1',
      'nivel': {'id': 'N2', 'nome': 'Grande Região'},
      'nome': 'Norte'},
     'serie': {'2000': '100.00', '2010': '100.00'}},
    {'localidade': {'id': '2',
      'nivel': {'id': 'N2', 'nome': 'Grande Região'},
      'nome': 'Nordeste'},
     'serie': {'2000': '100.00', '2010': '100.00'}},
    {'localidade': {'id': '3',
      'nivel': {'id': 'N2', 'nome': 'Grande Região'},
      'nome': 'Sudeste'},
     'serie': {'2000': '100.00', '2010': '100.00'}},
    {'localidade': {'id': '4',
      'nivel': {'id': 'N2', 'nome': 'Grande Região'},
      'nome': 'Su

In [13]:
make_df(get_request_json(url))

,id,variavel,unidade,resultados
0,849,Distribuição do rendimento nominal mensal das ...,%,"[{'classificacoes': [{'id': '2', 'nome': 'Sexo..."
1,842,Valor do rendimento nominal médio mensal das p...,Reais,"[{'classificacoes': [{'id': '2', 'nome': 'Sexo..."
2,843,Valor do rendimento nominal mediano mensal das...,Reais,"[{'classificacoes': [{'id': '2', 'nome': 'Sexo..."


In [14]:
make_df(get_request_json(url)).explode('resultados').explode('resultados')

,id,variavel,unidade,resultados
0,849,Distribuição do rendimento nominal mensal das ...,%,classificacoes
0,849,Distribuição do rendimento nominal mensal das ...,%,series
1,842,Valor do rendimento nominal médio mensal das p...,Reais,classificacoes
1,842,Valor do rendimento nominal médio mensal das p...,Reais,series
2,843,Valor do rendimento nominal mediano mensal das...,Reais,classificacoes
2,843,Valor do rendimento nominal mediano mensal das...,Reais,series


In [15]:
make_df(make_df(get_request_json(url)).explode('resultados')['resultados'][2]['series'])

,localidade_id,nivel_id,nivel_nome,localidade_nome,serie_2000,serie_2010
0,1,N2,Grande Região,Norte,220.00,510.00
1,2,N2,Grande Região,Nordeste,151.00,510.00
2,3,N2,Grande Região,Sudeste,369.00,800.00
3,4,N2,Grande Região,Sul,302.00,800.00
4,5,N2,Grande Região,Centro-Oeste,300.00,700.00


In [16]:
make_df(make_df(get_request_json(url)).explode('resultados')['resultados'][0]['classificacoes'])

,id,nome,categoria_6794,categoria_99852
0,2,Sexo,Total,NaN
1,12067,Classes acumuladas de percentual em ordem cres...,NaN,Até 100 %


### Código para extração de variáveis e valores

In [17]:
df_researches['id-agregado'].unique()

array(['102', '103', '105', '107', '108', '1134', '1161', '1206', '1209',
       '1211', '1285', '1286', '1287', '1288', '1289', '1290', '1292',
       '1294', '1296', '1298', '1301', '1302', '1309', '1310', '1311',
       '135', '136', '137', '1378', '1379', '138', '1381', '1382', '1383',
       '1384', '1385', '1386', '139', '1390', '1391', '1393', '1394',
       '1395', '1397', '1398', '140', '141', '142', '1423', '1424',
       '1425', '1426', '1427', '1428', '143', '1434', '1435', '1436',
       '1437', '1438', '1439', '144', '1440', '1442', '1444', '1446',
       '1447', '1449', '145', '1450', '1451', '1452', '1453', '1454',
       '1456', '1457', '1458', '146', '1460', '1461', '1462', '1463',
       '1464', '1465', '1466', '1467', '1468', '1469', '1470', '1471',
       '1472', '1473', '1474', '1475', '1476', '1477', '1478', '1479',
       '148', '1480', '1481', '1482', '1483', '1484', '1485', '1486',
       '1487', '1488', '1489', '149', '1490', '1491', '1492', '1493',
       '1

In [18]:
# this level gets the data divided by city in each of the 5 big regions
nivel = 'N6[N2[1,2,3,4,5]]'

# iterate over the agregados
for agregado in df_researches['id-agregado'].unique()[:1]:
    # generate a url to make the request using the specified level
    url = f'https://servicodados.ibge.gov.br/api/v3/agregados/{agregado}/variaveis?localidades={nivel}[all]' 
    print(url)
    # make the request and generate a dataframe 
    df = make_df(get_request_json(url))
    # get the results column and explode it
    df_results_json = df.explode('resultados')['resultados'][0]['series']
    # make a df from the results json
    df_results = make_df(df_results_json)
    # melt the df to transform serie (time period like year, semester, etc) in a column
    df_results = df_results.melt(id_vars=['localidade_id', 'nivel_id', 'nivel_nome', 'localidade_nome'], var_name = 'serie', value_name='valor')
    # remove the string 'serie_' from the serie column, so we have just the time period
    df_results['serie'] = df_results['serie'].apply(lambda x: x.replace('serie_', ''))
    print(df['variavel'])
    print('.')

https://servicodados.ibge.gov.br/api/v3/agregados/102/variaveis?localidades=N6[N2[1,2,3,4,5]][all]


KeyError: 'resultados'

In [ ]:
df

,id,variavel,unidade,resultados
0,1488,Mulheres de 10 anos ou mais de idade que vivia...,Pessoas,"[{'classificacoes': [{'id': '1', 'nome': 'Situ..."
1,1489,Mulheres de 10 anos ou mais de idade que vivia...,Pessoas,"[{'classificacoes': [{'id': '1', 'nome': 'Situ..."
2,1514,Filhos tidos pelas mulheres de 10 anos ou mais...,Pessoas,"[{'classificacoes': [{'id': '1', 'nome': 'Situ..."
3,1515,Filhos tidos nascidos vivos pelas mulheres de ...,Pessoas,"[{'classificacoes': [{'id': '1', 'nome': 'Situ..."
4,1525,Filhos tidos nascidos vivos pelas mulheres de ...,Pessoas,"[{'classificacoes': [{'id': '1', 'nome': 'Situ..."
5,1526,Filhos tidos nascidos mortos pelas mulheres de...,Pessoas,"[{'classificacoes': [{'id': '1', 'nome': 'Situ..."
6,1527,Filhos tidos pelas mulheres de 10 anos ou mais...,Pessoas,"[{'classificacoes': [{'id': '1', 'nome': 'Situ..."


In [49]:
url_w = 'https://servicodados.ibge.gov.br/api/v3/agregados/2065/periodos/-6/variaveis/96%7C1000096?localidades=N1[all]|N13[330101]|N6[N3[33]]&classificacao=471[0,12219]'
json_w = get_request_json(url_w)
json_w

[{'id': '96',
  'variavel': 'Domicílios particulares permanentes',
  'unidade': 'Unidades',
  'resultados': [{'classificacoes': [{'id': '471',
      'nome': 'Existência de água canalizada e forma de abastecimento',
      'categoria': {'0': 'Total'}}],
    'series': [{'localidade': {'id': '1',
       'nivel': {'id': 'N1', 'nome': 'Brasil'},
       'nome': 'Brasil'},
      'serie': {'2010': '57320555'}},
     {'localidade': {'id': '330101',
       'nivel': {'id': 'N13', 'nome': 'Região Metropolitana e Subdivisão'},
       'nome': 'Rio de Janeiro - Rio de Janeiro (RJ)'},
      'serie': {'2010': '3906323'}},
     {'localidade': {'id': '3300100',
       'nivel': {'id': 'N6', 'nome': 'Município'},
       'nome': 'Angra dos Reis - RJ'},
      'serie': {'2010': '53443'}},
     {'localidade': {'id': '3300159',
       'nivel': {'id': 'N6', 'nome': 'Município'},
       'nome': 'Aperibé - RJ'},
      'serie': {'2010': '3454'}},
     {'localidade': {'id': '3300209',
       'nivel': {'id': 'N6', 'no

In [50]:
make_df(json_w).explode('resultados')['resultados'][0]#['series']

0    {'classificacoes': [{'id': '471', 'nome': 'Exi...
0    {'classificacoes': [{'id': '471', 'nome': 'Exi...
Name: resultados, dtype: object

In [51]:
make_df(make_df(make_df(json_w).explode('resultados')['resultados']).explode('series')['series'])

,localidade_id,nivel_id,nivel_nome,localidade_nome,serie_2010
0,1,N1,Brasil,Brasil,57320555
1,330101,N13,Região Metropolitana e Subdivisão,Rio de Janeiro - Rio de Janeiro (RJ),3906323
2,3300100,N6,Município,Angra dos Reis - RJ,53443
3,3300159,N6,Município,Aperibé - RJ,3454
4,3300209,N6,Município,Araruama - RJ,35827
...,...,...,...,...,...
371,3306008,N6,Município,Três Rios - RJ,0.71
372,3306107,N6,Município,Valença - RJ,0.66
373,3306156,N6,Município,Varre-Sai - RJ,2.37
374,3306206,N6,Município,Vassouras - RJ,1.08


# Water

In [83]:
df_w_expl_1 = make_df(json_w).explode('resultados').reset_index(drop = True)
df_w_expl_1

,id,variavel,unidade,resultados
0,96,Domicílios particulares permanentes,Unidades,"{'classificacoes': [{'id': '471', 'nome': 'Exi..."
1,96,Domicílios particulares permanentes,Unidades,"{'classificacoes': [{'id': '471', 'nome': 'Exi..."
2,1000096,Domicílios particulares permanentes - percentu...,Percentual,"{'classificacoes': [{'id': '471', 'nome': 'Exi..."
3,1000096,Domicílios particulares permanentes - percentu...,Percentual,"{'classificacoes': [{'id': '471', 'nome': 'Exi..."


In [86]:
df_w_expl_2 = make_df(df_w_expl_1['resultados'])
df_w_expl_2 = pd.merge(df_w_expl_1, df_w_expl_2, left_index=True, right_index=True)
df_w_expl_2

,id,variavel,unidade,resultados,classificacoes,series
0,96,Domicílios particulares permanentes,Unidades,"{'classificacoes': [{'id': '471', 'nome': 'Exi...","[{'id': '471', 'nome': 'Existência de água can...","[{'localidade': {'id': '1', 'nivel': {'id': 'N..."
1,96,Domicílios particulares permanentes,Unidades,"{'classificacoes': [{'id': '471', 'nome': 'Exi...","[{'id': '471', 'nome': 'Existência de água can...","[{'localidade': {'id': '1', 'nivel': {'id': 'N..."
2,1000096,Domicílios particulares permanentes - percentu...,Percentual,"{'classificacoes': [{'id': '471', 'nome': 'Exi...","[{'id': '471', 'nome': 'Existência de água can...","[{'localidade': {'id': '1', 'nivel': {'id': 'N..."
3,1000096,Domicílios particulares permanentes - percentu...,Percentual,"{'classificacoes': [{'id': '471', 'nome': 'Exi...","[{'id': '471', 'nome': 'Existência de água can...","[{'localidade': {'id': '1', 'nivel': {'id': 'N..."


In [88]:
df_w_expl_3 = df_w_expl_2.explode('series')
df_w_expl_3 = df_w_expl_3.reset_index(drop = True)
df_w_expl_3

,id,variavel,unidade,resultados,classificacoes,series
0,96,Domicílios particulares permanentes,Unidades,"{'classificacoes': [{'id': '471', 'nome': 'Exi...","[{'id': '471', 'nome': 'Existência de água can...","{'localidade': {'id': '1', 'nivel': {'id': 'N1..."
1,96,Domicílios particulares permanentes,Unidades,"{'classificacoes': [{'id': '471', 'nome': 'Exi...","[{'id': '471', 'nome': 'Existência de água can...","{'localidade': {'id': '330101', 'nivel': {'id'..."
2,96,Domicílios particulares permanentes,Unidades,"{'classificacoes': [{'id': '471', 'nome': 'Exi...","[{'id': '471', 'nome': 'Existência de água can...","{'localidade': {'id': '3300100', 'nivel': {'id..."
3,96,Domicílios particulares permanentes,Unidades,"{'classificacoes': [{'id': '471', 'nome': 'Exi...","[{'id': '471', 'nome': 'Existência de água can...","{'localidade': {'id': '3300159', 'nivel': {'id..."
4,96,Domicílios particulares permanentes,Unidades,"{'classificacoes': [{'id': '471', 'nome': 'Exi...","[{'id': '471', 'nome': 'Existência de água can...","{'localidade': {'id': '3300209', 'nivel': {'id..."
...,...,...,...,...,...,...
371,1000096,Domicílios particulares permanentes - percentu...,Percentual,"{'classificacoes': [{'id': '471', 'nome': 'Exi...","[{'id': '471', 'nome': 'Existência de água can...","{'localidade': {'id': '3306008', 'nivel': {'id..."
372,1000096,Domicílios particulares permanentes - percentu...,Percentual,"{'classificacoes': [{'id': '471', 'nome': 'Exi...","[{'id': '471', 'nome': 'Existência de água can...","{'localidade': {'id': '3306107', 'nivel': {'id..."
373,1000096,Domicílios particulares permanentes - percentu...,Percentual,"{'classificacoes': [{'id': '471', 'nome': 'Exi...","[{'id': '471', 'nome': 'Existência de água can...","{'localidade': {'id': '3306156', 'nivel': {'id..."
374,1000096,Domicílios particulares permanentes - percentu...,Percentual,"{'classificacoes': [{'id': '471', 'nome': 'Exi...","[{'id': '471', 'nome': 'Existência de água can...","{'localidade': {'id': '3306206', 'nivel': {'id..."


In [93]:
df_w_expl_4 = pd.merge(df_w_expl_3, make_df(df_w_expl_3['series']), left_index=True, right_index=True)
df_w_expl_4

,id,variavel,unidade,resultados,classificacoes,series,localidade_id,nivel_id,nivel_nome,localidade_nome,serie_2010
0,96,Domicílios particulares permanentes,Unidades,"{'classificacoes': [{'id': '471', 'nome': 'Exi...","[{'id': '471', 'nome': 'Existência de água can...","{'localidade': {'id': '1', 'nivel': {'id': 'N1...",1,N1,Brasil,Brasil,57320555
1,96,Domicílios particulares permanentes,Unidades,"{'classificacoes': [{'id': '471', 'nome': 'Exi...","[{'id': '471', 'nome': 'Existência de água can...","{'localidade': {'id': '330101', 'nivel': {'id'...",330101,N13,Região Metropolitana e Subdivisão,Rio de Janeiro - Rio de Janeiro (RJ),3906323
2,96,Domicílios particulares permanentes,Unidades,"{'classificacoes': [{'id': '471', 'nome': 'Exi...","[{'id': '471', 'nome': 'Existência de água can...","{'localidade': {'id': '3300100', 'nivel': {'id...",3300100,N6,Município,Angra dos Reis - RJ,53443
3,96,Domicílios particulares permanentes,Unidades,"{'classificacoes': [{'id': '471', 'nome': 'Exi...","[{'id': '471', 'nome': 'Existência de água can...","{'localidade': {'id': '3300159', 'nivel': {'id...",3300159,N6,Município,Aperibé - RJ,3454
4,96,Domicílios particulares permanentes,Unidades,"{'classificacoes': [{'id': '471', 'nome': 'Exi...","[{'id': '471', 'nome': 'Existência de água can...","{'localidade': {'id': '3300209', 'nivel': {'id...",3300209,N6,Município,Araruama - RJ,35827
...,...,...,...,...,...,...,...,...,...,...,...
371,1000096,Domicílios particulares permanentes - percentu...,Percentual,"{'classificacoes': [{'id': '471', 'nome': 'Exi...","[{'id': '471', 'nome': 'Existência de água can...","{'localidade': {'id': '3306008', 'nivel': {'id...",3306008,N6,Município,Três Rios - RJ,0.71
372,1000096,Domicílios particulares permanentes - percentu...,Percentual,"{'classificacoes': [{'id': '471', 'nome': 'Exi...","[{'id': '471', 'nome': 'Existência de água can...","{'localidade': {'id': '3306107', 'nivel': {'id...",3306107,N6,Município,Valença - RJ,0.66
373,1000096,Domicílios particulares permanentes - percentu...,Percentual,"{'classificacoes': [{'id': '471', 'nome': 'Exi...","[{'id': '471', 'nome': 'Existência de água can...","{'localidade': {'id': '3306156', 'nivel': {'id...",3306156,N6,Município,Varre-Sai - RJ,2.37
374,1000096,Domicílios particulares permanentes - percentu...,Percentual,"{'classificacoes': [{'id': '471', 'nome': 'Exi...","[{'id': '471', 'nome': 'Existência de água can...","{'localidade': {'id': '3306206', 'nivel': {'id...",3306206,N6,Município,Vassouras - RJ,1.08


In [94]:
# df_w = make_df(make_df(make_df(json_w).explode('resultados')['resultados']).explode('series')['series'])
# df_w

In [104]:
df_w = df_w_expl_4[(df_w_expl_4['nivel_id'] == 'N6') 
                   & (df_w_expl_4['id'] == '1000096')
                   & (df_w_expl_4['serie_2010'] != '100.00')]
df_w

,id,variavel,unidade,resultados,classificacoes,series,localidade_id,nivel_id,nivel_nome,localidade_nome,serie_2010
284,1000096,Domicílios particulares permanentes - percentu...,Percentual,"{'classificacoes': [{'id': '471', 'nome': 'Exi...","[{'id': '471', 'nome': 'Existência de água can...","{'localidade': {'id': '3300100', 'nivel': {'id...",3300100,N6,Município,Angra dos Reis - RJ,0.85
285,1000096,Domicílios particulares permanentes - percentu...,Percentual,"{'classificacoes': [{'id': '471', 'nome': 'Exi...","[{'id': '471', 'nome': 'Existência de água can...","{'localidade': {'id': '3300159', 'nivel': {'id...",3300159,N6,Município,Aperibé - RJ,0.33
286,1000096,Domicílios particulares permanentes - percentu...,Percentual,"{'classificacoes': [{'id': '471', 'nome': 'Exi...","[{'id': '471', 'nome': 'Existência de água can...","{'localidade': {'id': '3300209', 'nivel': {'id...",3300209,N6,Município,Araruama - RJ,3.53
287,1000096,Domicílios particulares permanentes - percentu...,Percentual,"{'classificacoes': [{'id': '471', 'nome': 'Exi...","[{'id': '471', 'nome': 'Existência de água can...","{'localidade': {'id': '3300225', 'nivel': {'id...",3300225,N6,Município,Areal - RJ,1.37
288,1000096,Domicílios particulares permanentes - percentu...,Percentual,"{'classificacoes': [{'id': '471', 'nome': 'Exi...","[{'id': '471', 'nome': 'Existência de água can...","{'localidade': {'id': '3300233', 'nivel': {'id...",3300233,N6,Município,Armação dos Búzios - RJ,3.15
...,...,...,...,...,...,...,...,...,...,...,...
371,1000096,Domicílios particulares permanentes - percentu...,Percentual,"{'classificacoes': [{'id': '471', 'nome': 'Exi...","[{'id': '471', 'nome': 'Existência de água can...","{'localidade': {'id': '3306008', 'nivel': {'id...",3306008,N6,Município,Três Rios - RJ,0.71
372,1000096,Domicílios particulares permanentes - percentu...,Percentual,"{'classificacoes': [{'id': '471', 'nome': 'Exi...","[{'id': '471', 'nome': 'Existência de água can...","{'localidade': {'id': '3306107', 'nivel': {'id...",3306107,N6,Município,Valença - RJ,0.66
373,1000096,Domicílios particulares permanentes - percentu...,Percentual,"{'classificacoes': [{'id': '471', 'nome': 'Exi...","[{'id': '471', 'nome': 'Existência de água can...","{'localidade': {'id': '3306156', 'nivel': {'id...",3306156,N6,Município,Varre-Sai - RJ,2.37
374,1000096,Domicílios particulares permanentes - percentu...,Percentual,"{'classificacoes': [{'id': '471', 'nome': 'Exi...","[{'id': '471', 'nome': 'Existência de água can...","{'localidade': {'id': '3306206', 'nivel': {'id...",3306206,N6,Município,Vassouras - RJ,1.08


In [102]:
376/4

94.0